<a href="https://colab.research.google.com/github/takemr/intro-curriculum-3001/blob/master/dicomo_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#cloud
import os
import csv
from urllib.request import urlretrieve
from urllib.parse import urlparse
from zipfile import ZipFile
from tensorflow.contrib import rnn

###Air Qualityのダウンロード###
def download_file(url, output_dir, overwrite=False):
  # URLからファイル名を取得
  parse_result = urlparse(url)
  file_name = os.path.basename(parse_result.path)
  # 出力先ファイルパス
  destination = os.path.join(output_dir, file_name)

  # 無意味なダウンロードを防ぐため、上書き（overwrite）の指定か未ダウンロードの場合のみダウンロードを実施する
  if overwrite or not os.path.exists(destination):
    # 出力先ディレクトリの作成
    os.makedirs(output_dir)
    # ダウンロード
    urlretrieve(url, destination)

  return destination

zip_file = download_file('https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip', 'UCI_data/')
###ZipFile→AirQualityUCI.xlsx→air_quality
import pandas as pd

with ZipFile(zip_file) as z:
 with z.open('AirQualityUCI.xlsx') as f:
   air_quality = pd.read_excel(
     f,
     index_col=0, parse_dates={'DateTime': [0, 1]}, #1
     na_values=[-200.0],                            #2
     convert_float=False                            #3
   )

#初期のAirQuality　（欠損値は削除済）
#print("air_quality")
#print(air_quality)


#import cufflinks as cf
#cf.go_offline()

# 不要列の除去
target_columns = ['T', 'AH', 'PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)']
air_quality = air_quality[target_columns]
#不要列削除後のAirQuality
#print("air_quality")
#print(air_quality)






### TimeSriesDataSetクラスの定義 ###
import numpy as np

# 乱数シードの初期化（数値は何でもよい）
np.random.seed(12345)

# クラス定義
class TimeSeriesDataSet:

 def __init__(self, dataframe):
   self.feature_count = len(dataframe.columns)
   self.series_length = len(dataframe)
   self.series_data = dataframe.astype('float32')


 def __getitem__(self, n):
   return TimeSeriesDataSet(self.series_data[n])


 def __len__(self):
   return len(self.series_data)


 @property

 def times(self):
   return self.series_data.index


 def next_batch(self, length, batch_size):
   """
   連続したlength時間のデータおよび1時間の誤差測定用データを取得する。
   最後の1時間は最終出力データ。
   """
   max_start_index = len(self) - length
   design_matrix = []
   expectation = []
   while len(design_matrix) < batch_size:
     start_index = np.random.choice(max_start_index)
     end_index = start_index + length + 1
     values = self.series_data[start_index:end_index]
     if (values.count() == length + 1).all():  # 切り出したデータ中に欠損値がない
       train_data = values[:-1]
       true_value = values[-1:]
       design_matrix.append(train_data.as_matrix())
       expectation.append(np.reshape(true_value.as_matrix(), [self.feature_count]))
   return np.stack(design_matrix), np.stack(expectation)

 def append(self, data_point):
   dataframe = pd.DataFrame(data_point, columns=self.series_data.columns)
   self.series_data = self.series_data.append(dataframe)

 def tail(self, n):
   return TimeSeriesDataSet(self.series_data.tail(n))

 def as_array(self):
   return np.stack([self.series_data.as_matrix()])

 def mean(self):
   return self.series_data.mean()

 def std(self):
   return self.series_data.std()

 def standardize(self, mean=None, std=None):
   if mean is None:
     mean = self.mean()
   if std is None:
     std = self.std()
   return TimeSeriesDataSet((self.series_data - mean) / std)



###air_quality→dataset（TimeSeriesDataSetクラス）###
dataset = TimeSeriesDataSet(air_quality)
train_dataset = dataset[dataset.times.year < 2005]
leak_dataset = train_dataset[4<=train_dataset.times.month]
leak_dataset = leak_dataset[leak_dataset.times.month<=6]
test_dataset = dataset[dataset.times.year >= 2005]

#dataset
#print("dataset")
#print(dataset.series_data)

#generate original model
import tensorflow as tf
sess = tf.InteractiveSession()

# 再現性の確保のために乱数シードを固定
tf.set_random_seed(12345)

# パラメーター
# 学習時間長
SERIES_LENGTH = 72
# 特徴量数
FEATURE_COUNT = dataset.feature_count #6種類

# 入力（placeholderメソッドの引数は、データ型、テンソルのサイズ）
# 訓練データ

x = tf.placeholder(tf.float32, [None, SERIES_LENGTH, FEATURE_COUNT]) #batch_size（16） × 72 × 6
# 教師データ
y = tf.placeholder(tf.float32, [None, FEATURE_COUNT]) #batch_size（16） × 6

# RNNセルの作成
cell = tf.nn.rnn_cell.BasicLSTMCell(20)
initial_state = cell.zero_state(tf.shape(x)[0], dtype=tf.float32)
outputs, last_state = tf.nn.dynamic_rnn(cell, x, initial_state=initial_state, dtype=tf.float32)

# 全結合
# 重み
w = tf.Variable(tf.zeros([20, FEATURE_COUNT])) #w  20×6
# バイアス
b = tf.Variable([0.1] * FEATURE_COUNT) #6
# 最終出力（予測）
prediction = tf.matmul(last_state[-1], w) + b

# 損失関数（平均絶対誤差：MAE）と最適化（Adam）
loss = tf.reduce_mean(tf.map_fn(tf.abs, y - prediction))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# バッチサイズ
BATCH_SIZE = 16
# 学習回数
NUM_TRAIN =  10000 #ほんとは10000
# 学習中の出力頻度
OUTPUT_BY = 500

# 標準化
train_mean = train_dataset.mean()
train_std = train_dataset.std()
standardized_train_dataset = train_dataset.standardize() #train_dataset はこの形で今後処理を行う

#standardized_train_dataset
#print("standardized_train_dataset")
#print(standardized_train_dataset.series_data)

# 学習の実行
sess.run(tf.global_variables_initializer())
for i in range(NUM_TRAIN):

 batch = standardized_train_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE)

 mae, _ = sess.run([loss, optimizer], feed_dict={x: batch[0], y: batch[1]})

 if i % OUTPUT_BY == 0:
   print('step {:d}, error {:.2f}'.format(i, mae))

#f = open('leak.csv', 'w')
#cw = csv.writer(f)






###prediction###
def rnn_predict(input_dataset):
   # 標準化
   previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=train_mean, std=train_std)
   # 予測対象の時刻
   #print("aaa")
   #print(previous.times[0])
   #print(previous.times[-1])
   predict_time = previous.times[-1] + np.timedelta64(1, 'h')
   #print("predict_time")
   #print(predict_time)
   # 予測
   batch_x = previous.as_array()
   predict_data = prediction.eval({x: batch_x})
   
   # 結果のデータフレームを作成
   df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
   # 標準化の逆操作
   return train_mean + train_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in leak_dataset.times:
   predict_result = rnn_predict(air_quality[air_quality.index < current_time])
   #print("predict_result")
   #print(predict_result)
   #print("answear") #よう分からんしいらん
   #print(test_dataset[test_dataset.times == current_time].series_data)
   predict_air_quality = predict_air_quality.append(predict_result)

predict_air_quality.to_csv('leak.csv')

print("1正解leak")
print(leak_dataset.series_data)
print("モデルleak")
print(predict_air_quality)
#hyouka
absmean = np.mean(abs(leak_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )
      
#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))

"""
###One to Many
from functools import reduce
import re

def rnn_predict(input_dataset, ahead=1):
  # 標準化
  buffer = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=train_mean, std=train_std)
  # 予測対象の時刻
  last_time = buffer.times[-1]

  # 予測
  predict_data = []
  for i in range(0, ahead):
    batch_x = buffer.tail(SERIES_LENGTH).as_array()
    result_array = prediction.eval({x: batch_x})
    predict_data.append(result_array)
    buffer.append(result_array)
  predict_data = reduce(lambda acc, x: np.concatenate((acc, x)), predict_data)

  # インデックス時刻の作成
  hour = np.timedelta64(1, 'h')
  first_predict_time = input_dataset.index[-1] + hour
  index = np.arange(first_predict_time, first_predict_time + ahead * hour, step=hour)

  # 結果のデータフレームを作成
  df_standardized = pd.DataFrame(predict_data, columns=input_dataset.columns, index=index)
  # 標準化の逆操作
  return train_mean + train_std * df_standardized

predict_air_quality_1 = pd.DataFrame([], columns=air_quality.columns)
predict_air_quality_2 = pd.DataFrame([], columns=air_quality.columns)
predict_air_quality_3 = pd.DataFrame([], columns=air_quality.columns)
predict_air_quality_4 = pd.DataFrame([], columns=air_quality.columns)
predict_air_quality_5 = pd.DataFrame([], columns=air_quality.columns)
predict_air_quality_6 = pd.DataFrame([], columns=air_quality.columns)

for current_time in dataset.times[dataset.times.year >= 2005]:
  # 6時間先まで予測（結果は6レコード）
  predict_result = rnn_predict(air_quality[air_quality.index < current_time], 6)
  # 1時間先の予測レコード
  predict_air_quality_1 = predict_air_quality_1.append(predict_result[0:1])
  # 2時間先の予測レコード
  predict_air_quality_2 = predict_air_quality_2.append(predict_result[1:2])
  # 3時間先の予測レコード
  predict_air_quality_3 = predict_air_quality_3.append(predict_result[2:3])
  # 4時間先の予測レコード
  predict_air_quality_4 = predict_air_quality_4.append(predict_result[3:4])
  # 5時間先の予測レコード
  predict_air_quality_5 = predict_air_quality_5.append(predict_result[4:5])
  #6時間先の予測レコード
  predict_air_quality_6 = predict_air_quality_6.append(predict_result[5:6])

absmean = np.mean(abs(test_dataset.series_data - predict_air_quality_1))
print("1 hour later")
print(1-(absmean['T']/18.316054))
print(1-(absmean['AH']/1.025530))
print(1-(absmean['PT08.S1(CO)']/1099.707856))
print(1-(absmean['PT08.S2(NMHC)']/939.029205))
print(1-(absmean['PT08.S3(NOx)']/835.370973))
print(1-(absmean['PT08.S4(NO2)']/1456.143486))

absmean = np.mean(abs(test_dataset.series_data - predict_air_quality_2))
print("2 hour later")
print(1-(absmean['T']/18.316054))
print(1-(absmean['AH']/1.025530))
print(1-(absmean['PT08.S1(CO)']/1099.707856))
print(1-(absmean['PT08.S2(NMHC)']/939.029205))
print(1-(absmean['PT08.S3(NOx)']/835.370973))
print(1-(absmean['PT08.S4(NO2)']/1456.143486))

absmean = np.mean(abs(test_dataset.series_data - predict_air_quality_3))
print("3 hour later")
print(1-(absmean['T']/18.316054))
print(1-(absmean['AH']/1.025530))
print(1-(absmean['PT08.S1(CO)']/1099.707856))
print(1-(absmean['PT08.S2(NMHC)']/939.029205))
print(1-(absmean['PT08.S3(NOx)']/835.370973))
print(1-(absmean['PT08.S4(NO2)']/1456.143486))

absmean = np.mean(abs(test_dataset.series_data - predict_air_quality_4))
print("4 hour later")
print(1-(absmean['T']/18.316054))
print(1-(absmean['AH']/1.025530))
print(1-(absmean['PT08.S1(CO)']/1099.707856))
print(1-(absmean['PT08.S2(NMHC)']/939.029205))
print(1-(absmean['PT08.S3(NOx)']/835.370973))
print(1-(absmean['PT08.S4(NO2)']/1456.143486))

absmean = np.mean(abs(test_dataset.series_data - predict_air_quality_5))
print("5 hour later")
print(1-(absmean['T']/18.316054))
print(1-(absmean['AH']/1.025530))
print(1-(absmean['PT08.S1(CO)']/1099.707856))
print(1-(absmean['PT08.S2(NMHC)']/939.029205))
print(1-(absmean['PT08.S3(NOx)']/835.370973))
print(1-(absmean['PT08.S4(NO2)']/1456.143486))

absmean = np.mean(abs(test_dataset.series_data - predict_air_quality_6))
print("6 hour later")
print(1-(absmean['T']/18.316054))
print(1-(absmean['AH']/1.025530))
print(1-(absmean['PT08.S1(CO)']/1099.707856))
print(1-(absmean['PT08.S2(NMHC)']/939.029205))
print(1-(absmean['PT08.S3(NOx)']/835.370973))
print(1-(absmean['PT08.S4(NO2)']/1456.143486))
"""
sess.close()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:103: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


step 0, error 0.87
step 500, error 0.31
step 1000, error 0.25
step 1500, error 0.19
step 2000, error 0.23
step 2500, error 0.30
step 3000, error 0.16
step 3500, error 0.14
step 4000, error 0.22
step 4500, error 0.25
step 5000, error 0.19
step 5500, error 0.22
step 6000, error 0.17
step 6500, error 0.17
step 7000, error 0.20
step 7500, error 0.25
step 8000, error 0.26
step 8500, error 0.18
step 9000, error 0.24
step 9500, error 0.24


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


1正解leak
                             T        AH  PT08.S1(CO)  PT08.S2(NMHC)  \
DateTime                                                               
2004-04-01 00:00:00  11.950000  0.859344  1143.000000     825.250000   
2004-04-01 01:00:00  11.500000  0.865156  1043.750000     769.500000   
2004-04-01 02:00:00  10.675000  0.862988  1034.000000     715.500000   
2004-04-01 03:00:00   8.974999  0.839403   956.250000     713.250000   
2004-04-01 04:00:00  10.225000  0.829898   909.250000     615.000000   
2004-04-01 05:00:00  10.950000  0.832514   996.000000     648.250000   
2004-04-01 06:00:00   9.625000  0.824264  1153.500000     876.000000   
2004-04-01 07:00:00   9.500000  0.827329  1510.250000    1290.500000   
2004-04-01 08:00:00  11.875000  0.845475  1721.500000    1595.250000   
2004-04-01 09:00:00  16.225000  0.889207  1511.500000    1323.250000   
2004-04-01 10:00:00  19.150000  0.869266  1258.250000    1039.750000   
2004-04-01 11:00:00  19.725000  0.874324  1093.750000   

In [2]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      7 ?        00:00:01 node
     22 ?        00:00:04 node
     32 ?        00:00:08 jupyter-noteboo
    121 ?        00:00:00 tail
   1411 ?        00:10:56 python3
   1430 ?        00:00:00 python3
   1671 ?        00:00:00 ps


In [0]:
!kill 1411

In [1]:
import os
import csv
from urllib.request import urlretrieve
from urllib.parse import urlparse
from zipfile import ZipFile
from tensorflow.contrib import rnn

def download_file(url, output_dir, overwrite=False):
  # URLからファイル名を取得
  parse_result = urlparse(url)
  file_name = os.path.basename(parse_result.path)
  # 出力先ファイルパス
  destination = os.path.join(output_dir, file_name)

  # 無意味なダウンロードを防ぐため、上書き（overwrite）の指定か未ダウンロードの場合のみダウンロードを実施する
  if overwrite or not os.path.exists(destination):
    # 出力先ディレクトリの作成
    os.makedirs(output_dir)
    # ダウンロード
    urlretrieve(url, destination)

  return destination

zip_file = download_file('https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip', 'UCI_data/')

import pandas as pd

with ZipFile(zip_file) as z:
 with z.open('AirQualityUCI.xlsx') as f:
   air_quality = pd.read_excel(
     f,
     index_col=0, parse_dates={'DateTime': [0, 1]}, #1
     na_values=[-200.0],                            #2
     convert_float=False                            #3
   )

target_columns = ['T', 'AH', 'PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)']
air_quality = air_quality[target_columns]

print("air_quality")
print(type(air_quality))
print(air_quality)

import numpy as np

# 乱数シードの初期化（数値は何でもよい）
np.random.seed(12345)

# クラス定義
class TimeSeriesDataSet:

 def __init__(self, dataframe):
   self.feature_count = len(dataframe.columns)
   self.series_length = len(dataframe)
   self.series_data = dataframe.astype('float32')


 def __getitem__(self, n):
   return TimeSeriesDataSet(self.series_data[n])

 def __len__(self):
   return len(self.series_data)

 @property
 def times(self):
   return self.series_data.index

 def next_batch(self, length, batch_size, train):
   """
   連続したlength時間のデータおよび1時間の誤差測定用データを取得する。
   最後の1時間は最終出力データ。
   """
   max_start_index = len(self) - length
   design_matrix = []
   expectation = []
   t_pred = []
   while len(design_matrix) < batch_size:
     start_index = np.random.choice(max_start_index)
     end_index = start_index + length + 1
     values = self.series_data[start_index:end_index]
     t_values = train.series_data[start_index:end_index]
     if (values.count() == length + 1).all():  # 切り出したデータ中に欠損値がない
        train_data = values[:-1]
        true_value = values[-1:]
        teacher_value = t_values[-1:]
        design_matrix.append(train_data.as_matrix())
        expectation.append(np.reshape(true_value.as_matrix(), [self.feature_count]))
        t_pred.append(np.reshape(teacher_value.as_matrix(), [train.feature_count]))
   return np.stack(design_matrix), np.stack(expectation), np.stack(t_pred)

 def append(self, data_point):
   dataframe = pd.DataFrame(data_point, columns=self.series_data.columns)
   self.series_data = self.series_data.append(dataframe)

 def tail(self, n):
   return TimeSeriesDataSet(self.series_data.tail(n))

 def as_array(self):
   return np.stack([self.series_data.as_matrix()])

 def mean(self):
   return self.series_data.mean()

 def std(self):
   return self.series_data.std()

 def standardize(self, mean=None, std=None):
   if mean is None:
     mean = self.mean()
   if std is None:
     std = self.std()
   return TimeSeriesDataSet((self.series_data - mean) / std)


dataset = TimeSeriesDataSet(air_quality)
print("dataset")
print(dataset)
#train_dataset = dataset[dataset.times.year < 2005]
test_dataset = dataset[dataset.times.year >= 2005]

pre0_dataset = dataset[dataset.times.year < 2005]
pre_dataset = pre0_dataset[4<=pre0_dataset.times.month]
pre_dataset = pre_dataset[pre_dataset.times.month<=6]


#add
df = pd.read_csv('leak.csv', index_col=0, na_values=[-200.0])
#df = df.rename(columns={'Unnamed: 0': 'DateTime'})
print("df")
print (type(df))
print(df)


leakset = TimeSeriesDataSet(df)
print("leakset")
print(leakset)
train_dataset = leakset



import tensorflow as tf
sess = tf.InteractiveSession()

# 再現性の確保のために乱数シードを固定
tf.set_random_seed(12345)


# パラメーター
# 学習時間長
SERIES_LENGTH = 72
# 特徴量数
FEATURE_COUNT = dataset.feature_count #6

# 入力（placeholderメソッドの引数は、データ型、テンソルのサイズ）
# 訓練データ
x = tf.placeholder(tf.float32, [None, SERIES_LENGTH, FEATURE_COUNT])  #batch（16）, 72, 6
# 教師データ
y = tf.placeholder(tf.float32, [None, FEATURE_COUNT]) #batch（16）, 6
t = tf.placeholder(tf.float32, [None, FEATURE_COUNT]) #batch（16）, 6

# RNNセルの作成
cell = tf.nn.rnn_cell.BasicRNNCell(20)
initial_state = cell.zero_state(tf.shape(x)[0], dtype=tf.float32)
outputs, last_state = tf.nn.dynamic_rnn(cell, x, initial_state=initial_state, dtype=tf.float32)

# 全結合
# 重み
w = tf.Variable(tf.zeros([20, FEATURE_COUNT]))
# バイアス
b = tf.Variable([0.1] * FEATURE_COUNT)
# 最終出力（予測）
#prediction = tf.matmul(last_state[-1], w) + b
prediction = tf.matmul(last_state, w) + b

# 損失関数（平均絶対誤差：MAE）と最適化（Adam）
loss_0 = tf.reduce_mean(tf.map_fn(tf.abs, y - prediction))
L2loss_s = tf.nn.l2_loss(prediction - y)
L2loss_t = tf.nn.l2_loss(t - y)
L2loss_s2 = (tf.nn.l2_loss(prediction - y))**2
L2loss_t2 = (tf.nn.l2_loss(t - y))**2
L2loss_2 = ((tf.nn.l2_loss(prediction - t))**2)/2

optimizer0 = tf.train.AdamOptimizer().minimize(loss_0)
optimizer = tf.train.AdamOptimizer().minimize(L2loss_s)
optimizer2 = tf.train.AdamOptimizer().minimize(L2loss_s2)
optimizer3 = tf.train.AdamOptimizer().minimize(L2loss_2)


# バッチサイズ
BATCH_SIZE = 16
# 学習回数
NUM_TRAIN =  5000
# 学習中の出力頻度
OUTPUT_BY = 1000




train_mean = train_dataset.mean()
train_std = train_dataset.std()
standardized_train_dataset = train_dataset.standardize() #train_dataset はこの形で今後処理を行う

pre_mean = pre_dataset.mean()
pre_std = pre_dataset.std()
standardized_pre_dataset = pre_dataset.standardize() #pre_dataset はこの形で今後処理を行う


########################################
# 学習の実行     平均絶対誤差のみ
sess.run(tf.global_variables_initializer())

for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s = sess.run(loss_0, feed_dict={x: batch[0], y: batch[1], t:batch[2]})
    
 sess.run(optimizer0, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  label only-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))









########################################
# 学習の実行    La
sess.run(tf.global_variables_initializer())

for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_2, L2loss_t2], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 sess.run(optimizer3, feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  La-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))








########################################
# 学習の実行    Lb0 m = 5
sess.run(tf.global_variables_initializer())

m=5
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s2, L2loss_t2], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer2, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb0  m = 5-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))








########################################
# 学習の実行  Lb0 m = 50
sess.run(tf.global_variables_initializer())

m=50
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s2, L2loss_t2], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer2, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb0 m = 50-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))









########################################
# 学習の実行    Lb0 m = 100
sess.run(tf.global_variables_initializer())

m=100
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s2, L2loss_t2], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer2, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb0 m = 100-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))








########################################
# 学習の実行   Lb0  m = 500
sess.run(tf.global_variables_initializer())

m=500
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s2, L2loss_t2], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer2, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb0 m = 500-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))


########################################
# 学習の実行   Lb0  m = 0.5
sess.run(tf.global_variables_initializer())

m=0.5
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s2, L2loss_t2], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer2, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb0 m = 500-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))











########################################
# 学習の実行    Lb m = 5
sess.run(tf.global_variables_initializer())

m=5
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s, L2loss_t], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb  m = 5-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )

#abs = abs(test_dataset.series_data - predict_air_quality)

#print(tf.reduce_mean(tf.map_fn(test_dataset.series_data - predict_air_quality)))





############################################ 
#学習の実行　　Lb m = 50

sess.run(tf.global_variables_initializer())

m=50
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s, L2loss_t], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb m = 50-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )



########################################################
#学習の実行　　Lb m = 100

sess.run(tf.global_variables_initializer())

m=100
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s, L2loss_t], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb  m = 100-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )






########################################################
#学習の実行　　　Lb m = 500
sess.run(tf.global_variables_initializer())

m=500
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s, L2loss_t], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb  m = 500-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )





########################################################
#学習の実行　　　Lb m = 0.5
sess.run(tf.global_variables_initializer())

m = 0.5
for i in range(NUM_TRAIN):

 batch = standardized_pre_dataset.next_batch(SERIES_LENGTH, BATCH_SIZE, standardized_train_dataset)
 
 norm_s, norm_t = sess.run([L2loss_s, L2loss_t], feed_dict={x: batch[0], y: batch[1], t:batch[2]})
 
 diff = norm_s - norm_t
 
    
 if norm_s + m > norm_t:
   sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
 
 if i % OUTPUT_BY == 0:
  print('step {:d}, error {:.2f}'.format(i, norm_s))
  print(norm_t)

  
  
  
  
def rnn_predict(input_dataset):
 # 標準化
 previous = TimeSeriesDataSet(input_dataset).tail(SERIES_LENGTH).standardize(mean=pre_mean, std=pre_std)
 # 予測対象の時刻
 predict_time = previous.times[-1] + np.timedelta64(1, 'h')
  
 # 予測
 batch_x = previous.as_array()
 predict_data = prediction.eval({x: batch_x})

 # 結果のデータフレームを作成
 df_standardized  = pd.DataFrame(predict_data, columns=input_dataset.columns, index=[predict_time])
 # 標準化の逆操作
 return pre_mean + pre_std * df_standardized

predict_air_quality = pd.DataFrame([], columns=air_quality.columns)

for current_time in test_dataset.times:
 predict_result = rnn_predict(air_quality[air_quality.index < current_time])
 predict_air_quality = predict_air_quality.append(predict_result)

#hyouka
absmean = np.mean(abs(test_dataset.series_data - predict_air_quality))
print(absmean)

acc_t = 1-(absmean['T']/18.316054)
print(acc_t)

acc_ah = 1-(absmean['AH']/1.025530)
print(acc_ah)

acc_co = 1-(absmean['PT08.S1(CO)']/1099.707856)
print(acc_co)

acc_nmhc = 1-(absmean['PT08.S2(NMHC)']/939.029205)
print(acc_nmhc)

acc_nox = 1-(absmean['PT08.S3(NOx)']/835.370973)
print(acc_nox)

acc_no2 = 1-(absmean['PT08.S4(NO2)']/1456.143486)
print(acc_no2)

print("acc  Lb  m = 500-------------------------------------------------------")
print( (acc_t + acc_ah + acc_co + acc_nmhc + acc_nox + acc_no2)/6 )


sess.close()

air_quality
<class 'pandas.core.frame.DataFrame'>
                             T        AH  PT08.S1(CO)  PT08.S2(NMHC)  \
DateTime                                                               
2004-03-10 18:00:00  13.600000  0.757754  1360.000000    1045.500000   
2004-03-10 19:00:00  13.300000  0.725487  1292.250000     954.750000   
2004-03-10 20:00:00  11.900000  0.750239  1402.000000     939.250000   
2004-03-10 21:00:00  11.000000  0.786713  1375.500000     948.250000   
2004-03-10 22:00:00  11.150000  0.788794  1272.250000     835.500000   
2004-03-10 23:00:00  11.175000  0.784772  1197.000000     750.250000   
2004-03-11 00:00:00  11.325000  0.760312  1185.000000     689.500000   
2004-03-11 01:00:00  10.675000  0.770238  1136.250000     672.000000   
2004-03-11 02:00:00  10.650000  0.764819  1094.000000     608.500000   
2004-03-11 03:00:00  10.250000  0.751657  1009.750000     560.750000   
2004-03-11 04:00:00  10.075000  0.746495  1011.000000     526.750000   
2004-03-11 05:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


step 0, error 0.73
step 1000, error 0.31
step 2000, error 0.24
step 3000, error 0.19
step 4000, error 0.29


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


T                  3.365541
AH                 0.077758
PT08.S1(CO)       77.093079
PT08.S2(NMHC)    104.196312
PT08.S3(NOx)     122.116989
PT08.S4(NO2)     238.586273
dtype: float32
0.8162518270512916
0.9241773215462918
0.9298967646792174
0.8890382627123045
0.8538170548382914
0.836151948288591
acc  label only-------------------------------------------------------
0.874888863185998
step 0, error 718.34
step 1000, error 17.03
step 2000, error 3.69
step 3000, error 3.37
step 4000, error 1.81
T                  3.360172
AH                 0.069217
PT08.S1(CO)       83.761826
PT08.S2(NMHC)    113.570480
PT08.S3(NOx)     136.013809
PT08.S4(NO2)     295.658112
dtype: float32
0.8165449680521517
0.9325062748343216
0.9238326569147257
0.8790554332687878
0.8371815473601552
0.7969581195707347
acc  La-------------------------------------------------------
0.864346500000146
step 0, error 2887.03
17.556393
step 1000, error 103.04
65.01578
step 2000, error 67.32
26.567234
step 3000, error 41.07
36.905

In [2]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      6 ?        00:00:01 node
     23 ?        00:00:03 node
     33 ?        00:00:07 jupyter-noteboo
    126 ?        00:00:00 tail
   1183 ?        00:52:48 python3
   1217 ?        00:00:00 python3
   1790 ?        00:00:00 ps


In [0]:
!kill  1183

88.89


5月
0.9562003119735011
0.966117349018336
0.9483239553704763
0.9290228464660454
0.9229807231117118
0.9489187915725145